## Astr-119-Final-Project

In [ ]:
import numpy as np
import sep

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

from astropy.io import fits

In [ ]:
from astropy.utils.data import download_file
image_file = download_file('https://github.com/kbarbary/sep/blob/v1.0.x/data/image.fits', cache=True)

In [ ]:
hdu_list = fits.open(image_file)
hdu_list.info()